In [1]:
import pandas as pd
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
'''data columns are: Index(['Day', 'Month', 'Year', 'States/UTs', 'Rice', 'Wheat', 'Atta (Wheat)',
       'Gram Dal', 'Tur/Arhar Dal', 'Urad Dal', 'Moong Dal', 'Masoor Dal',
       'Sugar', 'Milk @', 'Groundnut Oil (Packed)', 'Mustard Oil (Packed)',
       'Vanaspati (Packed)', 'Soya Oil (Packed)', 'Sunflower Oil (Packed)',
       'Palm Oil (Packed)', 'Gur', 'Tea Loose', 'Salt Pack (Iodised)',
       'Potato', 'Onion', 'Tomato'],
      dtype='object')'''

"data columns are: Index(['Day', 'Month', 'Year', 'States/UTs', 'Rice', 'Wheat', 'Atta (Wheat)',\n       'Gram Dal', 'Tur/Arhar Dal', 'Urad Dal', 'Moong Dal', 'Masoor Dal',\n       'Sugar', 'Milk @', 'Groundnut Oil (Packed)', 'Mustard Oil (Packed)',\n       'Vanaspati (Packed)', 'Soya Oil (Packed)', 'Sunflower Oil (Packed)',\n       'Palm Oil (Packed)', 'Gur', 'Tea Loose', 'Salt Pack (Iodised)',\n       'Potato', 'Onion', 'Tomato'],\n      dtype='object')"

In [26]:
# Load the data from the CSV file
df = pd.read_csv(r"data\cleansing\filled\newdata.csv")

In [27]:
df

,Day,Month,Year,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
0,1,1,2016,29.50,30.50,26.67,70.50,155.00,160.00,105.75,...,79.25,88.000000,92.25,59.25,43.00,215.00,16.25,20.50,23.25,29.50
1,2,1,2016,29.60,31.40,27.50,71.40,154.60,161.20,108.20,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,20.40,23.60,32.00
2,3,1,2016,29.60,31.40,27.50,71.40,154.00,161.20,108.20,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,20.40,23.60,35.00
3,4,1,2016,29.60,31.40,27.50,71.40,151.60,160.20,107.20,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,21.40,23.00,41.40
4,5,1,2016,29.50,31.75,27.50,71.75,151.50,161.50,107.50,...,80.25,88.000000,91.75,58.75,45.75,215.00,17.00,21.25,22.75,41.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,1,9,2024,56.12,44.88,52.88,99.76,172.82,137.65,120.76,...,126.29,120.706538,115.82,95.59,57.53,286.23,24.41,49.00,53.47,23.76
3142,2,9,2024,57.31,45.73,54.06,99.75,173.53,139.00,122.47,...,126.69,125.605161,116.00,95.78,57.19,283.88,24.16,48.97,53.84,23.94
3143,3,9,2024,57.53,45.80,54.75,99.88,172.23,138.19,121.97,...,126.38,125.397419,116.06,95.91,57.34,283.96,24.19,48.88,54.41,24.53
3144,4,9,2024,57.29,45.79,55.19,99.48,172.30,138.90,122.03,...,127.16,125.459032,116.16,95.97,56.84,287.13,24.07,49.32,55.45,26.58


In [5]:
print(df.shape)

(3146, 25)


In [ ]:
df=df[df['Year']>2022]
print(df.shape)

In [6]:
# Convert date columns to a datetime object
df['Date'] = pd.to_datetime(df[['Day', 'Month', 'Year']])
df.set_index('Date', inplace=True)

In [7]:
df

,Day,Month,Year,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,1,1,2016,29.50,30.50,26.67,70.50,155.00,160.00,105.75,...,79.25,88.000000,92.25,59.25,43.00,215.00,16.25,20.50,23.25,29.50
2016-01-02,2,1,2016,29.60,31.40,27.50,71.40,154.60,161.20,108.20,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,20.40,23.60,32.00
2016-01-03,3,1,2016,29.60,31.40,27.50,71.40,154.00,161.20,108.20,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,20.40,23.60,35.00
2016-01-04,4,1,2016,29.60,31.40,27.50,71.40,151.60,160.20,107.20,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,21.40,23.00,41.40
2016-01-05,5,1,2016,29.50,31.75,27.50,71.75,151.50,161.50,107.50,...,80.25,88.000000,91.75,58.75,45.75,215.00,17.00,21.25,22.75,41.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01,1,9,2024,56.12,44.88,52.88,99.76,172.82,137.65,120.76,...,126.29,120.706538,115.82,95.59,57.53,286.23,24.41,49.00,53.47,23.76
2024-09-02,2,9,2024,57.31,45.73,54.06,99.75,173.53,139.00,122.47,...,126.69,125.605161,116.00,95.78,57.19,283.88,24.16,48.97,53.84,23.94
2024-09-03,3,9,2024,57.53,45.80,54.75,99.88,172.23,138.19,121.97,...,126.38,125.397419,116.06,95.91,57.34,283.96,24.19,48.88,54.41,24.53


In [8]:
# Select the crops for prediction
crops = ['Rice', 'Wheat', 'Atta (Wheat)',
       'Gram Dal', 'Tur/Arhar Dal', 'Urad Dal', 'Moong Dal', 'Masoor Dal',
       'Sugar', 'Milk @', 'Groundnut Oil (Packed)', 'Mustard Oil (Packed)',
       'Vanaspati (Packed)', 'Soya Oil (Packed)', 'Sunflower Oil (Packed)',
       'Palm Oil (Packed)', 'Gur', 'Tea Loose', 'Salt Pack (Iodised)',
       'Potato', 'Onion', 'Tomato']

In [9]:
print(len(crops))

22


In [10]:
# Create a dictionary to store the last known data for each state
last_known_data = {}
last_known_data["tn"] = df.iloc[-1][crops].values

In [11]:
print(df)

            Day  Month  Year   Rice  Wheat  Atta (Wheat)  Gram Dal  \
Date                                                                 
2016-01-01    1      1  2016  29.50  30.50         26.67     70.50   
2016-01-02    2      1  2016  29.60  31.40         27.50     71.40   
2016-01-03    3      1  2016  29.60  31.40         27.50     71.40   
2016-01-04    4      1  2016  29.60  31.40         27.50     71.40   
2016-01-05    5      1  2016  29.50  31.75         27.50     71.75   
...         ...    ...   ...    ...    ...           ...       ...   
2024-09-01    1      9  2024  56.12  44.88         52.88     99.76   
2024-09-02    2      9  2024  57.31  45.73         54.06     99.75   
2024-09-03    3      9  2024  57.53  45.80         54.75     99.88   
2024-09-04    4      9  2024  57.29  45.79         55.19     99.48   
2024-09-05    5      9  2024  57.47  45.64         53.77    100.30   

            Tur/Arhar Dal  Urad Dal  Moong Dal  ...  Vanaspati (Packed)  \
Date          

In [12]:
# Prepare data for LSTM
data = df[crops].values
print(data.shape)
data

(3146, 22)


array([[29.5 , 30.5 , 26.67, ..., 20.5 , 23.25, 29.5 ],
       [29.6 , 31.4 , 27.5 , ..., 20.4 , 23.6 , 32.  ],
       [29.6 , 31.4 , 27.5 , ..., 20.4 , 23.6 , 35.  ],
       ...,
       [57.53, 45.8 , 54.75, ..., 48.88, 54.41, 24.53],
       [57.29, 45.79, 55.19, ..., 49.32, 55.45, 26.58],
       [57.47, 45.64, 53.77, ..., 49.9 , 56.  , 27.13]])

In [13]:
# Scale the data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)
print(scaled_data)

[[0.062744   0.08675535 0.         ... 0.1292732  0.08383234 0.1611815 ]
 [0.06553263 0.13880856 0.02547575 ... 0.12640046 0.08662675 0.17992353]
 [0.06553263 0.13880856 0.02547575 ... 0.12640046 0.08662675 0.20241397]
 ...
 [0.84439487 0.97165992 0.86187845 ... 0.94455616 0.33261477 0.12392233]
 [0.83770218 0.97108155 0.87538367 ... 0.95719621 0.34091816 0.1392908 ]
 [0.8427217  0.96240602 0.83179865 ... 0.97385809 0.34530938 0.14341405]]


In [14]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(scaled_data, test_size=0.2,shuffle=False)
train_data

array([[0.062744  , 0.08675535, 0.        , ..., 0.1292732 , 0.08383234,
        0.1611815 ],
       [0.06553263, 0.13880856, 0.02547575, ..., 0.12640046, 0.08662675,
        0.17992353],
       [0.06553263, 0.13880856, 0.02547575, ..., 0.12640046, 0.08662675,
        0.20241397],
       ...,
       [0.66229782, 0.64256796, 0.68201351, ..., 0.74059178, 0.17053892,
        0.11327686],
       [0.66229782, 0.64256796, 0.68201351, ..., 0.76299914, 0.16878244,
        0.13164405],
       [0.66229782, 0.64256796, 0.68201351, ..., 0.74691181, 0.16702595,
        0.1357673 ]])

In [15]:
print("Train data shape: ", train_data.shape)
print("Test data shape: ", test_data.shape)

Train data shape:  (2516, 22)
Test data shape:  (630, 22)


In [16]:
# Create the function to create the dataset
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [17]:
# Set look_back period (number of previous days to consider)
look_back = 7
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

In [18]:
X_train

array([[[0.062744  , 0.08675535, 0.        , ..., 0.1292732 ,
         0.08383234, 0.1611815 ],
        [0.06553263, 0.13880856, 0.02547575, ..., 0.12640046,
         0.08662675, 0.17992353],
        [0.06553263, 0.13880856, 0.02547575, ..., 0.12640046,
         0.08662675, 0.20241397],
        ...,
        [0.062744  , 0.15905147, 0.02547575, ..., 0.15081873,
         0.07984032, 0.24739486],
        [0.05995538, 0.12724118, 0.02547575, ..., 0.14363689,
         0.08822355, 0.27438339],
        [0.05995538, 0.12724118, 0.02547575, ..., 0.16087331,
         0.08343313, 0.28337956]],

       [[0.06553263, 0.13880856, 0.02547575, ..., 0.12640046,
         0.08662675, 0.17992353],
        [0.06553263, 0.13880856, 0.02547575, ..., 0.12640046,
         0.08662675, 0.20241397],
        [0.06553263, 0.13880856, 0.02547575, ..., 0.15512784,
         0.08183633, 0.25039358],
        ...,
        [0.05995538, 0.12724118, 0.02547575, ..., 0.14363689,
         0.08822355, 0.27438339],
        [0.0

In [19]:
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

In [20]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(X_train.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2508/2508 - 14s - 5ms/step - loss: 0.0041
Epoch 2/100
2508/2508 - 10s - 4ms/step - loss: 0.0015
Epoch 3/100
2508/2508 - 9s - 4ms/step - loss: 0.0012
Epoch 4/100
2508/2508 - 9s - 4ms/step - loss: 0.0011
Epoch 5/100
2508/2508 - 10s - 4ms/step - loss: 0.0010
Epoch 6/100
2508/2508 - 10s - 4ms/step - loss: 9.5507e-04
Epoch 7/100
2508/2508 - 9s - 4ms/step - loss: 8.9342e-04
Epoch 8/100
2508/2508 - 11s - 4ms/step - loss: 8.5502e-04
Epoch 9/100
2508/2508 - 12s - 5ms/step - loss: 8.0431e-04
Epoch 10/100
2508/2508 - 11s - 4ms/step - loss: 7.8927e-04
Epoch 11/100
2508/2508 - 11s - 5ms/step - loss: 7.5961e-04
Epoch 12/100
2508/2508 - 11s - 5ms/step - loss: 7.2823e-04
Epoch 13/100
2508/2508 - 11s - 4ms/step - loss: 6.8866e-04
Epoch 14/100
2508/2508 - 11s - 4ms/step - loss: 6.8332e-04
Epoch 15/100
2508/2508 - 11s - 4ms/step - loss: 6.5109e-04
Epoch 16/100
2508/2508 - 10s - 4ms/step - loss: 6.4654e-04
Epoch 17/100
2508/2508 - 11s - 4ms/step - loss: 6.3642e-04
Epoch 18/100
2508/2508 - 12s 

In [21]:
# save the model
model.save('model.keras')
# load the model


In [22]:
# Save the scaler
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
look_back

In [23]:
# Scale the data to be between 0 and 1
#scaler = MinMaxScaler(feature_range=(0, 1))
last_week_data = df[crops].iloc[-7:].values 
scaled_last_week = scaler.transform(last_week_data) 
reshaped_last_week = np.reshape(scaled_last_week, (1, look_back, len(crops))) 

In [24]:
# Predict
prediction_scaled = model.predict(reshaped_last_week)
prediction = scaler.inverse_transform(prediction_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step


In [25]:
# Create DataFrame for 6/9/2024 predictions
predictions_6_9_2024 = pd.DataFrame(data=prediction, columns=crops)

In [ ]:
predictions_6_9_2024

In [ ]:
# Predict for all states using the last known data
for state, state_data in last_known_data.items():
    scaled_state_data = scaler.transform(state_data.reshape(1,-1))
    reshaped_state_data = np.reshape(scaled_state_data, (1, 1, len(crops)))

    state_prediction_scaled = model.predict(reshaped_state_data)
    state_prediction = scaler.inverse_transform(state_prediction_scaled)

    predictions_6_9_2024.loc[state] = state_prediction[0]

In [ ]:
# Display the predictions
print(predictions_6_9_2024)

# --------------------------------------------------

In [26]:
# Predict for all states using a rolling window of past data
predictions_by_state = {}  # Dictionary to store predictions for each state

In [ ]:
for state in df['States/UTs'].unique():
    state_data = df[df['States/UTs'] == state][crops].iloc[-look_back:].values # Last 'look_back' days of data
    scaled_state_data = scaler.transform(state_data)
    reshaped_state_data = np.reshape(scaled_state_data, (1, look_back, len(crops)))

    state_prediction_scaled = model.predict(reshaped_state_data)
    state_prediction = scaler.inverse_transform(state_prediction_scaled)
    predictions_by_state[state] = state_prediction[0] # Store prediction for the state


In [ ]:
predictions_df = pd.DataFrame(predictions_by_state).T # Convert to DataFrame for better display
predictions_df.columns = crops # Add column names
print(predictions_df)

In [ ]:
# test the model
# Load the model
model = tensorflow.keras.models.load_model('model.keras')

# Load the scaler
scaler = joblib.load('scaler.pkl')

# it hase one state value

